<img align="right" width="200" height="200" src="https://static.tildacdn.com/tild6236-6337-4339-b337-313363643735/new_logo.png">

# Spark Dataframes I
**Сергей Гришаев**  
serg.grishaev@gmail.com  

## На этом занятии
+ Сравнение RDD API и DataFrame API 
+ Базовые функции
+ Очистка данных
+ Агрегаты
+ Кеширование 
+ Репартиционирование
+ Встроенные функции
+ Пользовательские функции
+ Соединения
+ Оконные функции

## Сравнение RDD API и DataFrame API 

### Типы данных
**RDD**: низкоуревная распределенная коллекция данных любого типа  
**DF**: таблица со схемой, состоящей из колонок разных типов, описанных в `org.apache.spark.sql.types`  

### Обработка данных
**RDD**: сериализуемые функции  
**DF**: кодогенерация SQL > Java код  

### Функции и алгоритмы
**RDD**: нет ограничений  
**DF**: ограничен SQL операторами, функциями `org.apache.spark.sql.functions` и пользовательскими функциями  

### Источники данных
**RDD**: каждый источник имеет свое API  
**DF**: единое API для всех источников 

### Производительность
**RDD**: напрямую зависит от качества кода
**DF**: встроенные механизмы оптимизации SQL запроса


### Потоковая обработка данных
**RDD**: устаревший DStreams  
**DF**: активно развивающийся Structured Streaming


### Выводы:
+ На текущий момент RDD является низкоуровневым API, которое постепенно уходит "под капот" Apache Spark
+ DF API представляет собой библиотеку для обработки данных с использованием SQL примитивов

## Базовые функции

Создать dataframe можно на основе:
+ локальной коллекции
+ файлов
+ базы данных

In [1]:
import org.apache.spark.sql.DataFrame

val cityList: Vector[String] = Vector("Moscow", "Paris", "Madrid", "London", "New York")

// метод toDF изначально отсутствует у Vector[T], он добавляется через import spark.implicits._
val df: DataFrame = cityList.toDF

cityList = Vector(Moscow, Paris, Madrid, London, New York)
df = [value: string]


[value: string]

У любого DF есть схема:

In [2]:
df.printSchema

root
 |-- value: string (nullable = true)



Посмотреть содержимое DF можно с помощью метода `show()`:

In [3]:
df.show()

+--------+
|   value|
+--------+
|  Moscow|
|   Paris|
|  Madrid|
|  London|
|New York|
+--------+



Также можно вывести содержимое в вертикальной ориентации - это удобно при большое количестве столбцов:

In [4]:
df.show(numRows = 20, truncate = 100, vertical=true)

-RECORD 0---------
 value | Moscow   
-RECORD 1---------
 value | Paris    
-RECORD 2---------
 value | Madrid   
-RECORD 3---------
 value | London   
-RECORD 4---------
 value | New York 



Подсчет количества элементов в DF с помощью `count()`:

In [5]:
df.count

5

Отфильтровать данные можно с помощью метода `filter`. В отличие от RDD, он принимает SQL выражение:

In [16]:
// Требует наличия import spark.implicits._

df.filter(('value !== "Moscow").and( ('value !== "Moscow"))).show

+--------+
|   value|
+--------+
|   Paris|
|  Madrid|
|  London|
|New York|
+--------+



In [11]:
df.filter('value.!==("Moscow")).show

+--------+
|   value|
+--------+
|   Paris|
|  Madrid|
|  London|
|New York|
+--------+



In [21]:
$"value" toString ()

value

In [17]:
// Требует наличия import spark.implicits._

df.filter($"value" === "Moscow").show

+------+
| value|
+------+
|Moscow|
+------+



In [22]:
// sugar free & type safe
// Три знака равно здесь используются, тк на самом деле это метод,
// применяемый к колонке org.apache.spark.sql.Column

import org.apache.spark.sql.functions.col

df.filter(col("value") === "Moscow").show

+------+
| value|
+------+
|Moscow|
+------+



In [27]:
// легко ошибиться и получить ошибку в рантайме
import org.apache.spark.sql.functions.lit

val qE = 
    df.filter("value = 'Moscow'")
        .localCheckpoint
        .withColumn("foo", lit("foo"))
        .withColumn("bar", lit("bar"))
        .queryExecution

qE = 


== Parsed Logical Plan ==
Project [value#1, foo#68, bar AS bar#71]
+- Project [value#1, foo AS foo#68]
   +- LogicalRDD [value#1], false
== Analyzed Logical Plan ==
value: string, foo: string, bar: string
Project [value#1, foo#68, bar AS bar#71]
+- Project [value#1, foo AS foo#68]
   +- LogicalRDD [value#1], false
== Optimized Logical Plan ==
Project [value#1, foo AS foo#68, bar AS bar#71]
+- LogicalRDD [value#1], false
== Physical Plan ==
*(1) Project [value#1, foo AS foo#68, bar AS bar#71]
+- Scan ExistingRDD[value#1]


In [28]:
qE.logical

Project [value#1, foo#68, bar AS bar#71]
+- Project [value#1, foo AS foo#68]
   +- LogicalRDD [value#1], false


In [32]:
val sqlParser = spark.sessionState.sqlParser
sqlParser.parsePlan("""SELECT FOO FROM BAR""")

sqlParser = org.apache.spark.sql.execution.SparkSqlParser@5bd91b0f


lastException: Throwable = null
'Project ['FOO]
+- 'UnresolvedRelation `BAR`


In [33]:
qE.analyzed

Project [value#1, foo#68, bar AS bar#71]
+- Project [value#1, foo AS foo#68]
   +- LogicalRDD [value#1], false


In [34]:
qE.optimizedPlan // CollapseProject

Project [value#1, foo AS foo#68, bar AS bar#71]
+- LogicalRDD [value#1], false


In [35]:
qE.executedPlan

*(1) Project [value#1, foo AS foo#68, bar AS bar#71]
+- Scan ExistingRDD[value#1]


In [39]:
import org.apache.spark.sql.catalyst.InternalRow
import org.apache.spark.rdd.RDD

val rdd: RDD[InternalRow] = qE.toRdd
rdd

rdd = SQLExecutionRDD[24] at toRdd at <console>:34


SQLExecutionRDD[24] at toRdd at <console>:34

In [40]:
// промежуточный вариант между col и обычной строкой
// expr также может использоваться для вызова SQL builtin функций, 
// отсутствующих в org.apache.spark.sql.functions

import org.apache.spark.sql.functions.expr

df.filter(expr("value = 'Moscow'")).show

+------+
| value|
+------+
|Moscow|
+------+



Добавить новую колонку можно с помощью метода `withColumn`. Необходимо помнить, что данный метод, как и другие, является трансформацией и не изменяет оригинальный DF, а создает новый.

In [41]:
import org.apache.spark.sql.functions.upper
df.withColumn("upperCity", upper('value)).show

+--------+---------+
|   value|upperCity|
+--------+---------+
|  Moscow|   MOSCOW|
|   Paris|    PARIS|
|  Madrid|   MADRID|
|  London|   LONDON|
|New York| NEW YORK|
+--------+---------+



Аналогичный результат получить, используя метод `select`. Данный метод может быть использован не только для выбора определенных колонок, но и для создания новых.

In [50]:
val withUpper = df.select('value, upper('value).alias("upperCity"))
withUpper.show

+--------+---------+
|   value|upperCity|
+--------+---------+
|  Moscow|   MOSCOW|
|   Paris|    PARIS|
|  Madrid|   MADRID|
|  London|   LONDON|
|New York| NEW YORK|
+--------+---------+



withUpper = [value: string, upperCity: string]


[value: string, upperCity: string]

Если передать `col("*")` в `select`, то вы получите DF со всеми колонками. Это полезно, когда вы не знаете список всех колонок (например вы получили его через API), но вам нужно их все выбрать и добавить новую колонку. Это можно сделать следующим образом:

In [51]:
// методы name, as и alias часто являются взаимозаменяемыми

import org.apache.spark.sql.functions._

withUpper.select(
    col("*"), 
    lower($"value").name("lowerCity"), 
    (length('value) + 1).as("length"),
    lit("foo").alias("bar")).show

+--------+---------+---------+------+---+
|   value|upperCity|lowerCity|length|bar|
+--------+---------+---------+------+---+
|  Moscow|   MOSCOW|   moscow|     7|foo|
|   Paris|    PARIS|    paris|     6|foo|
|  Madrid|   MADRID|   madrid|     7|foo|
|  London|   LONDON|   london|     7|foo|
|New York| NEW YORK| new york|     9|foo|
+--------+---------+---------+------+---+



При необходимости в `select` можно передать список колонок, используя обычные строки:

In [53]:
withUpper.select("value", "upperCity").show

+--------+---------+
|   value|upperCity|
+--------+---------+
|  Moscow|   MOSCOW|
|   Paris|    PARIS|
|  Madrid|   MADRID|
|  London|   LONDON|
|New York| NEW YORK|
+--------+---------+



Удалить колонку из DF можно с помощью метода `drop`:

In [55]:
// drop не будет выдавать ошибку, если будет указана несуществующая колонка

withUpper.drop("upperCity", "abraKadabra", "2323").show

+--------+
|   value|
+--------+
|  Moscow|
|   Paris|
|  Madrid|
|  London|
|New York|
+--------+



In [56]:
// Жертвуем отказоустойчивостью для повышения производительности
withUpper.localCheckpoint.drop("upperCity", "abraKadabra").explain(true)

== Parsed Logical Plan ==
Project [value#1]
+- LogicalRDD [value#1, upperCity#126], false

== Analyzed Logical Plan ==
value: string
Project [value#1]
+- LogicalRDD [value#1, upperCity#126], false

== Optimized Logical Plan ==
Project [value#1]
+- LogicalRDD [value#1, upperCity#126], false

== Physical Plan ==
*(1) Project [value#1]
+- Scan ExistingRDD[value#1,upperCity#126]


In [59]:
(col("abc") === col("cde")) alias "123"

(abc = cde) AS `123`

### Выводы:
+ методы `filter` и `select` принимают в качестве аргументов колонки [org.apache.spark.sql.Column](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Column). Это может быть либо ссылка на существующую колонку, либо функцию из [org.apache.spark.sql.functions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$)
+ любые трансформации возвращают новый DF, не меняя существующий
+ тип [org.apache.spark.sql.Column](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Column) играет важную роль в DF API - на его основе создаются ссылки на существующие колонки, а также функции, принимающие [org.apache.spark.sql.Column](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Column) и возвращающие [org.apache.spark.sql.Column](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Column). По этой причине обычное сравнение `==` не будет работать в DF API, тк `filter` принимает [org.apache.spark.sql.Column](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Column), а не `Boolean`
+ Класс DataFrame в последних версиях Spark представляет собой `org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]`, поэтому его описание следует искать в [org.apache.spark.sql.Dataset](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset)

## Очистка данных

Одной из задач обработки данных является их очистка. DF API содержит класс функций "not available", описанный в пакете [org.apache.spark.sql.DataFrameNaFunctions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.DataFrameNaFunctions). В данном пакете есть три функции:
+ `na.drop`
+ `na.fill`
+ `na.replace`

Для демонстрации работы данных функций создадим новый датасет:

In [61]:
import org.apache.spark.sql.Column
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Dataset

val testData =
"""{ "name":"Moscow", "country":"Rossiya", "continent": "Europe", "population": 12380664}
{ "name":"Madrid", "country":"Spain" }
{ "name":"Paris", "country":"France", "continent": "Europe", "population" : 2196936}
{ "name":"Berlin", "country":"Germany", "continent": "Europe", "population": 3490105}
{ "name":"Barselona", "country":"Spain", "continent": "Europe" }
{ "name":"Cairo", "country":"Egypt", "continent": "Africa", "population": 11922948 }
{ "name":"Cairo", "country":"Egypt", "continent": "Africa", "population": 11922948 }
{ "name":"New York, "country":"USA","""

// Создаем DF из одной строки и добавляем данные в виде новой колонки
val raw = spark.range(0,1).select(lit(testData).alias("value"))

// Создаем новую колонку, разибая наши данные по \n
val jsonStrings: Column = split(col("value"), "\n").alias("value")

// Используем функцию explode для того, чтобы развернуть массив мехом наружу и используем темную магию 
// для превращения DataFrame в Dataset[String]
val splited: Dataset[String] = raw.select(explode(jsonStrings)).as[String]

// splited.show(numRows = 10, truncate = false)


// Создаем новый датафре... датасет, в котором наши JSON строки будут распарсены
val df: Dataset[Row] = spark.read.json(splited)
df.printSchema
df.show

root
 |-- _corrupt_record: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- country: string (nullable = true)
 |-- name: string (nullable = true)
 |-- population: long (nullable = true)

+--------------------+---------+-------+---------+----------+
|     _corrupt_record|continent|country|     name|population|
+--------------------+---------+-------+---------+----------+
|                null|   Europe|Rossiya|   Moscow|  12380664|
|                null|     null|  Spain|   Madrid|      null|
|                null|   Europe| France|    Paris|   2196936|
|                null|   Europe|Germany|   Berlin|   3490105|
|                null|   Europe|  Spain|Barselona|      null|
|                null|   Africa|  Egypt|    Cairo|  11922948|
|                null|   Africa|  Egypt|    Cairo|  11922948|
|{ "name":"New Yor...|     null|   null|     null|      null|
+--------------------+---------+-------+---------+----------+



testData = 
raw = [value: string]


{ "name":"Moscow", "country":"Rossiya", "continent": "Europe", "population": 12380664}
{ "name":"Madrid", "country":"Spain" }
{ "name":"Paris", "country":"France", "continent": "Europe", "population" : 2196936}
{ "name":"Berlin", "country":"Germany", "continent": "Europe", "population": 3490105}
{ "name":"Barselona", "country":"Spain", "continent": "Europe" }
{ "name":"Cairo", "country":"Egypt", "continent": "Africa", "population": 11922948 }
{ "name":"Cairo", "country":"Egypt", "continent": "Africa", "population": 11922948 }
{ "name":"New York, "country":"USA",
jsonStrings:...


[value: string]

Для очистки датасета:
+ удалим строку с навалидным JSON, сохраним ее в отдельное место
+ удалим дубликаты
+ заполним `null`ы в колонках
+ исправим `Rossiya` на `Russia`

In [62]:
val corruptData = df.select(col("_corrupt_record")).na.drop("all").collect

corruptData = Array([{ "name":"New York, "country":"USA",])


Array([{ "name":"New York, "country":"USA",])

In [65]:
val fillData: Map[String, Any] = Map("continent" -> "Undefined", "population" -> 0)
val replaceData: Map[Any, Any] = Map("Rossiya" -> "Russia")

val cleanData = 
    df
    .drop(col("_corrupt_record"))
    .na.drop("all")
    .na.fill(fillData)
    .na.replace("country", replaceData)
    .dropDuplicates
    .localCheckpoint


cleanData.show

+---------+-------+---------+----------+
|continent|country|     name|population|
+---------+-------+---------+----------+
|   Europe| France|    Paris|   2196936|
|   Europe|Germany|   Berlin|   3490105|
|Undefined|  Spain|   Madrid|         0|
|   Africa|  Egypt|    Cairo|  11922948|
|   Europe|  Spain|Barselona|         0|
|   Europe| Russia|   Moscow|  12380664|
+---------+-------+---------+----------+



fillData = Map(continent -> Undefined, population -> 0)
replaceData = Map(Rossiya -> Russia)
cleanData = [continent: string, country: string ... 2 more fields]


[continent: string, country: string ... 2 more fields]

### Выводы:
+ DF API обладает удобным API для очистки данных, позволяющим разработчику сконцентрироваться разработчику на бизнес логике, а не на написании функций для обработки всех возможных исключительных ситуаций
+ метод `spark.read.json` позволяет читать не только файлы, но и `Dataset[String]`, содержащие JSON строки.

## Агрегаты
Посчитаем суммарное население и количество городов с разбивкой по континентам:

In [66]:
val aggCount = cleanData.groupBy('continent).count
aggCount.show

+---------+-----+
|continent|count|
+---------+-----+
|   Europe|    4|
|   Africa|    1|
|Undefined|    1|
+---------+-----+



aggCount = [continent: string, count: bigint]


[continent: string, count: bigint]

In [67]:
val aggSum = cleanData.groupBy('continent).sum("population")
aggSum.show

+---------+---------------+
|continent|sum(population)|
+---------+---------------+
|   Europe|       18067705|
|   Africa|       11922948|
|Undefined|              0|
+---------+---------------+



aggSum = [continent: string, sum(population): bigint]


[continent: string, sum(population): bigint]

Для того, чтобы совместить несколько агрегатов в одном DF, мы можем использовать метод `agg()`. Данный метод позволяет использовать любые `Aggregate functions` из пакета [org.apache.spark.sql.functions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$)

In [68]:
val agg = cleanData.groupBy('continent).agg(count("*").alias("count"), sum("population").alias("sumPop"))
agg.show

+---------+-----+--------+
|continent|count|  sumPop|
+---------+-----+--------+
|   Europe|    4|18067705|
|   Africa|    1|11922948|
|Undefined|    1|       0|
+---------+-----+--------+



agg = [continent: string, count: bigint ... 1 more field]


[continent: string, count: bigint ... 1 more field]

С помощью агрегатов мы можем выполнять такие действия, как, например, `collect_list` и `collect_set`. Стоит отметить, что колонки в Spark могут иметь не только скалярные типы, но и структуры, словари и массивы:

In [69]:
val aggList = cleanData.groupBy('continent).agg(collect_list("country").alias("countries"))
aggList.printSchema
aggList.show(numRows = 10, truncate = 100, vertical = true)

root
 |-- continent: string (nullable = false)
 |-- countries: array (nullable = true)
 |    |-- element: string (containsNull = true)

-RECORD 0-------------------------------------
 continent | Europe                           
 countries | [France, Germany, Spain, Russia] 
-RECORD 1-------------------------------------
 continent | Africa                           
 countries | [Egypt]                          
-RECORD 2-------------------------------------
 continent | Undefined                        
 countries | [Spain]                          



aggList = [continent: string, countries: array<string>]


[continent: string, countries: array<string>]

Используя методы `struct` и `to_json`, мы можем превратить произвольный набор колонок в JSON строку. Этот методы часто используется перед отправкой данных в Kafka

In [70]:
val withStruct = aggList.select(struct('continent, 'countries).alias("s"))
withStruct.printSchema

withStruct.show(10, false)
withStruct.select(col("`s`.`continent`"))

root
 |-- s: struct (nullable = false)
 |    |-- continent: string (nullable = false)
 |    |-- countries: array (nullable = true)
 |    |    |-- element: string (containsNull = true)

+------------------------------------------+
|s                                         |
+------------------------------------------+
|[Europe, [France, Germany, Spain, Russia]]|
|[Africa, [Egypt]]                         |
|[Undefined, [Spain]]                      |
+------------------------------------------+



withStruct = [s: struct<continent: string, countries: array<string>>]


[continent: string]

In [71]:
withStruct.withColumn("s", to_json('s)).show(10, false)

+------------------------------------------------------------------------+
|s                                                                       |
+------------------------------------------------------------------------+
|{"continent":"Europe","countries":["France","Germany","Spain","Russia"]}|
|{"continent":"Africa","countries":["Egypt"]}                            |
|{"continent":"Undefined","countries":["Spain"]}                         |
+------------------------------------------------------------------------+



Если необходимо превратить все колонки DF в JSON String, можно воспользоваться функций `toJSON`:

In [72]:
val jString: Dataset[String] = aggList.toJSON
jString.show(5, false)

+------------------------------------------------------------------------+
|value                                                                   |
+------------------------------------------------------------------------+
|{"continent":"Europe","countries":["France","Germany","Spain","Russia"]}|
|{"continent":"Africa","countries":["Egypt"]}                            |
|{"continent":"Undefined","countries":["Spain"]}                         |
+------------------------------------------------------------------------+



jString = [value: string]


[value: string]

Если нам необходимо создать колонки из значений текущих колонок, мы можем воспользоваться функцией `pivot`

In [73]:
cleanData.show

+---------+-------+---------+----------+
|continent|country|     name|population|
+---------+-------+---------+----------+
|   Europe| France|    Paris|   2196936|
|   Europe|Germany|   Berlin|   3490105|
|Undefined|  Spain|   Madrid|         0|
|   Africa|  Egypt|    Cairo|  11922948|
|   Europe|  Spain|Barselona|         0|
|   Europe| Russia|   Moscow|  12380664|
+---------+-------+---------+----------+



In [74]:
cleanData.groupBy(col("country")).pivot("continent").agg(sum("population")).show

+-------+--------+--------+---------+
|country|  Africa|  Europe|Undefined|
+-------+--------+--------+---------+
| Russia|    null|12380664|     null|
|Germany|    null| 3490105|     null|
| France|    null| 2196936|     null|
|  Spain|    null|       0|        0|
|  Egypt|11922948|    null|     null|
+-------+--------+--------+---------+



In [76]:
cleanData.groupBy(col("country")).pivot("continent")

Name: Unknown Error
Message: <console>:57: error: value show is not a member of org.apache.spark.sql.RelationalGroupedDataset
       cleanData.groupBy(col("country")).pivot("continent").show()
                                                            ^

StackTrace: 

### Выводы:
+ DF API позволяет строить большое количество агрегатов. При этом необходимо помнить, что операции `groupBy`, `cube`, `rollup` возвращают [org.apache.spark.sql.RelationalGroupedDataset](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.RelationalGroupedDataset), к которому затем необходимо применить одну из функций агрегации - `count`, `sum`, `agg` и т. п.
+ При вычислении агрегатов необходимо помнить, что эта операция требует перемешивания данных между воркерами, что, в случае перекошенных данных, может привести к OOM на воркере.

## Кеширование
По умолчанию при применении каждого действия Spark пересчитывает весь граф, что может негативно сказать на производительности приложения. Для демонстрации возьмем датасет [Airport Codes](https://datahub.io/core/airport-codes)  

In [77]:
val csvOptions = Map("header" -> "true", "inferSchema" -> "true")
val airports = spark.read.options(csvOptions).csv("/tmp/airport-codes_csv.csv")
airports.printSchema
airports.show(numRows = 1, truncate = 100, vertical = true)

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)

-RECORD 0------------------------------------------
 ident        | 00A                                
 type         | heliport                           
 name         | Total Rf Heliport                  
 elevation_ft | 11                                 
 continent    | NA                                 
 iso_country  | US                                 
 iso_region   | US-PA                              
 municipality | Bensalem                           
 gps_code     | 00A                 

csvOptions = Map(header -> true, inferSchema -> true)
airports = [ident: string, type: string ... 10 more fields]


[ident: string, type: string ... 10 more fields]

In [78]:
airports.rdd.getNumPartitions

2

In [79]:
airports.show(10)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|     US-AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|       NA|         US|     

Посчитаем несколько агрегатов. Несмотря на то, что `onlyRu` является общим для всех действий, он пересчитывается при вызове каждого действия.

In [83]:
val onlyRuAndHigh = airports.filter('iso_country === "RU" and 'elevation_ft > 1000)
onlyRuAndHigh.show(numRows = 1, truncate = 100, vertical = true)
onlyRuAndHigh.count
onlyRuAndHigh.collect
onlyRuAndHigh.groupBy('municipality).count.orderBy('count.desc).na.drop("any").explain()

-RECORD 0-----------------------------
 ident        | RU-0006               
 type         | closed                
 name         | Arabatuk Air Base     
 elevation_ft | 2280                  
 continent    | EU                    
 iso_country  | RU                    
 iso_region   | RU-CHI                
 municipality | Daurija               
 gps_code     | null                  
 iata_code    | null                  
 local_code   | ZA2N                  
 coordinates  | 117.098999, 50.223801 
only showing top 1 row

== Physical Plan ==
*(3) Sort [count#914L DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(count#914L DESC NULLS LAST, 200)
   +- *(2) Filter AtLeastNNulls(n, municipality#581,count#914L)
      +- *(2) HashAggregate(keys=[municipality#581], functions=[count(1)])
         +- Exchange hashpartitioning(municipality#581, 200)
            +- *(1) HashAggregate(keys=[municipality#581], functions=[partial_count(1)])
               +- *(1) Project [municipality#581]

onlyRuAndHigh = [ident: string, type: string ... 10 more fields]


[ident: string, type: string ... 10 more fields]

Для решения этой проблемы следует использовать методы `cache`, либо `persist`. Данные методы сохраняют состояние графа после первого действия, и следующие обращаются к нему. Разница между методами заключается в том, что `persist` позволяет выбрать, куда сохранить данные, а `cache` использует значение по умолчанию. В текущей версии Spark это [StorageLevel.MEMORY_ONLY](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence). Важно помнить, что данный кеш не предназначен для обмена данными между разными Spark приложения - он является внутренним для приложения. После того, как работа с данными окончена, необходимо выполнить `unpersist` для очистки памяти

In [85]:
onlyRuAndHigh.cache
onlyRuAndHigh.count
// при вычислении count данные будут помещены в cache
onlyRuAndHigh.show(numRows = 1, truncate = 100, vertical = true)
onlyRuAndHigh.collect
onlyRuAndHigh.groupBy('municipality).count.orderBy('count.desc).na.drop("any").explain()

onlyRuAndHigh.unpersist

-RECORD 0-----------------------------
 ident        | RU-0006               
 type         | closed                
 name         | Arabatuk Air Base     
 elevation_ft | 2280                  
 continent    | EU                    
 iso_country  | RU                    
 iso_region   | RU-CHI                
 municipality | Daurija               
 gps_code     | null                  
 iata_code    | null                  
 local_code   | ZA2N                  
 coordinates  | 117.098999, 50.223801 
only showing top 1 row

== Physical Plan ==
*(3) Sort [count#1514L DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(count#1514L DESC NULLS LAST, 200)
   +- *(2) Filter AtLeastNNulls(n, municipality#581,count#1514L)
      +- *(2) HashAggregate(keys=[municipality#581], functions=[count(1)])
         +- Exchange hashpartitioning(municipality#581, 200)
            +- *(1) HashAggregate(keys=[municipality#581], functions=[partial_count(1)])
               +- InMemoryTableScan [municipal

[ident: string, type: string ... 10 more fields]

### Выводы:
+ Использование `cache` и `persist` позволяет существенно сократить время обработки данных, однако следует помнить и об увеличении потребляемой памяти на воркерах

## Репартиционирование
RDD и DF являются представляют собой классы, описывающие распределенные коллекции данных. Они (коллекции) разбиты на крупные блоки, которые называются партициями. В графе вычисления, который называется в Spark DAG (Direct Acyclic Graph), есть три основных компонента - `job`, `stage`, `task`.

`job` представляет собой весь граф целиком, от момента создания DF, до применения `action` к нему. Состоит из одной или более `stage`. Когда возникает необходимость сделать `shuffle` данных, Spark создает новый `stage`. Каждый `stage` состоит из большого количества `task`. `task` это базовая операция над данными. Одновременно Spark выполняет N `task`, которые обрабатывают N партиций, где N - это суммарное число доступных потоков на всех воркерах.

Исходя из этого, важно обеспечивать:
+ достаточное количество партиций для распределения нагрузки по всем воркерам
+ равномерное распределение данных между партициями

Создадим датасет с перекосом данных:

In [86]:
import org.apache.spark.sql.functions._

val skewColumn = when(col("id") < 900, lit(0)).otherwise(lit(1))

val skewDf = spark.range(0,1000).repartition(10, skewColumn)

def printItemPerPartition[T](ds: Dataset[T]): Unit = {
    ds.mapPartitions { x => Iterator(x.length) }
    .withColumnRenamed("value", "itemPerPartition")
    .show(50, false)
}

printItemPerPartition[java.lang.Long](skewDf)

+----------------+
|itemPerPartition|
+----------------+
|0               |
|900             |
|0               |
|100             |
|0               |
|0               |
|0               |
|0               |
|0               |
|0               |
+----------------+



skewColumn = CASE WHEN (id < 900) THEN 0 ELSE 1 END
skewDf = [id: bigint]


printItemPerPartition: [T](ds: org.apache.spark.sql.Dataset[T])Unit


[id: bigint]

Любые операции с таким датасетом будут работать медленно, т.к.
+ если суммарное количество потоков на всех воркерах больше 10, то в один момент времени работать будут максимум 10, остальные будут простаивать
+ из 10 партицийи только в 2 есть данные и это означает, что только 2 потока будут обрабатывать данные, при этом из-за перекоса данных между ними (900 vs 100) первый станет bottleneck'ом

Обычно перекошенные датасеты возникают после вычисления агрегатов, оконных функций и соединений, но также могут возникать и при чтении источников.

Для устранения проблемы перекоса данных, следует использовать метод `repartition`:

In [90]:
// здесь мы передаем только новое количество партиций и Spark выполнит RoundRobinPartitioning
val repartitionedDf = skewDf.repartition(20)
repartitionedDf.explain()
printItemPerPartition[java.lang.Long](repartitionedDf)

== Physical Plan ==
Exchange RoundRobinPartitioning(20)
+- *(1) Range (0, 1000, step=1, splits=2)
+----------------+
|itemPerPartition|
+----------------+
|50              |
|50              |
|50              |
|50              |
|50              |
|50              |
|50              |
|50              |
|50              |
|50              |
|50              |
|50              |
|50              |
|50              |
|50              |
|50              |
|50              |
|50              |
|50              |
|50              |
+----------------+



repartitionedDf = [id: bigint]


[id: bigint]

In [89]:
// здесь мы добавляем к числу партиций колонку, по которой необходимо сделать репартиционирование,
// поэтому Spark выполнит HashPartitioning
val repartitionedDf = skewDf.repartition(20, col("id"))
repartitionedDf.explain()
printItemPerPartition[java.lang.Long](repartitionedDf)

== Physical Plan ==
Exchange hashpartitioning(id#1643L, 20)
+- *(1) Range (0, 1000, step=1, splits=2)
+----------------+
|itemPerPartition|
+----------------+
|37              |
|61              |
|48              |
|59              |
|47              |
|54              |
|45              |
|58              |
|55              |
|55              |
|56              |
|46              |
|45              |
|46              |
|49              |
|64              |
|44              |
|39              |
|40              |
|52              |
+----------------+



repartitionedDf = [id: bigint]


[id: bigint]

<img align="right" width="200" height="200" src="https://pngimage.net/wp-content/uploads/2018/06/соленья-png-4.png">

### Соленья
Часто при вычислении агрегатов приходится работать с перекошенными данными:

In [91]:
airports.printSchema

airports.groupBy('type).count.orderBy('count.desc).show

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)

+--------------+-----+
|          type|count|
+--------------+-----+
| small_airport|34808|
|      heliport|12028|
|medium_airport| 4537|
|        closed| 4378|
| seaplane_base| 1030|
| large_airport|  616|
|   balloonport|   24|
+--------------+-----+



Поскольку при вычислении агрегата происходит неявный `HashPartitioning` по ключу (ключам) агрегата, то при выполнении определенных условий происходит нехватка памяти на воркере, которую нельзя исправить, не изменив подход к построению агрегата.

Один из вариантов устранение - соление ключей:

In [92]:
val saltModTen = pmod(round((rand() * 100), 0), lit(10)).cast("int")

val salted = airports.withColumn("salt", saltModTen)
salted.show(numRows = 1, truncate = 200, vertical = true)

-RECORD 0------------------------------------------
 ident        | 00A                                
 type         | heliport                           
 name         | Total Rf Heliport                  
 elevation_ft | 11                                 
 continent    | NA                                 
 iso_country  | US                                 
 iso_region   | US-PA                              
 municipality | Bensalem                           
 gps_code     | 00A                                
 iata_code    | null                               
 local_code   | 00A                                
 coordinates  | -74.93360137939453, 40.07080078125 
 salt         | 3                                  
only showing top 1 row



saltModTen = CAST(pmod(round((rand(-5330693255816678478) * 100), 0), 10) AS INT)
salted = [ident: string, type: string ... 11 more fields]


[ident: string, type: string ... 11 more fields]

Это позволяет нам существенно снизить объем данных в каждой партиции (30к vs 3к):

In [93]:
val firstStep = salted.groupBy('type, 'salt).count()

firstStep.orderBy('count.desc).show(200, false)

+--------------+----+-----+
|type          |salt|count|
+--------------+----+-----+
|small_airport |3   |3616 |
|small_airport |2   |3558 |
|small_airport |5   |3518 |
|small_airport |0   |3490 |
|small_airport |6   |3479 |
|small_airport |9   |3449 |
|small_airport |8   |3433 |
|small_airport |7   |3431 |
|small_airport |1   |3427 |
|small_airport |4   |3407 |
|heliport      |4   |1255 |
|heliport      |7   |1233 |
|heliport      |6   |1225 |
|heliport      |2   |1220 |
|heliport      |3   |1206 |
|heliport      |0   |1192 |
|heliport      |1   |1192 |
|heliport      |9   |1178 |
|heliport      |5   |1164 |
|heliport      |8   |1163 |
|medium_airport|5   |491  |
|medium_airport|0   |489  |
|medium_airport|8   |468  |
|closed        |9   |457  |
|medium_airport|4   |456  |
|closed        |6   |455  |
|closed        |2   |454  |
|medium_airport|6   |454  |
|medium_airport|1   |447  |
|closed        |1   |447  |
|medium_airport|3   |446  |
|closed        |3   |446  |
|medium_airport|9   

firstStep = [type: string, salt: int ... 1 more field]


[type: string, salt: int ... 1 more field]

Вторым шагом мы делаем еще один агрегат, суммируя предыдущие значения `count`:

In [94]:
val secondStep = firstStep.groupBy('type).agg(sum("count").alias("count"))

secondStep.orderBy('count.desc).show(200, false)

+--------------+-----+
|type          |count|
+--------------+-----+
|small_airport |34808|
|heliport      |12028|
|medium_airport|4537 |
|closed        |4378 |
|seaplane_base |1030 |
|large_airport |616  |
|balloonport   |24   |
+--------------+-----+



secondStep = [type: string, count: bigint]


[type: string, count: bigint]

Несмотря на то, что мы сделали две группировки вместо одной, распределение данных по воркерам было более равномерным, что позволило избежать OOM на воркерах.

### Выводы:
+ Партиционирование - важный аспект распределенных вычислений, от которого напрямую зависит стабильность и скорость вычислений
+ В Spark всегда работает правило 1 TASK = 1 THREAD (= vCore = 1 virtual CPU core) = 1 PARTITION
+ Репартиционирование и соление данных позволяет решить проблему перекоса данных и вычислений
+ Важно помнить, что репартиционирование использует дисковую и сетевую подсистемы - обмен данными происходит **по сети**, а результат записывается **на диск**, что может стать узким местом при выполнении репартиционирования

## Встроенные функции
Помимо базовых SQL операторов, в Spark существует большой набор встроенных функций:
+ API методы из [org.apache.spark.sql.functions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$)
+ [SQL built-in functions](https://spark.apache.org/docs/latest/api/sql/index.html)

In [96]:
val df = spark.range(0,10)

// используем org.apache.spark.sql.functions
val newCol: Column = pmod(col("id"), lit(2))
df.withColumn("pmod", newCol).show

+---+----+
| id|pmod|
+---+----+
|  0|null|
|  1|   0|
|  2|   0|
|  3|   0|
|  4|   0|
|  5|   0|
|  6|   0|
|  7|   0|
|  8|   0|
|  9|   0|
+---+----+



df = [id: bigint]
newCol = pmod(id, id)


pmod(id, id)

In [97]:
import org.apache.spark.sql.functions.expr

// используем SQL built-in functions
val newCol: Column = expr("""pmod(id, 2)""")
df.withColumn("pmod", newCol).show

+---+----+
| id|pmod|
+---+----+
|  0|   0|
|  1|   1|
|  2|   0|
|  3|   1|
|  4|   0|
|  5|   1|
|  6|   0|
|  7|   1|
|  8|   0|
|  9|   1|
+---+----+



newCol = pmod(id, 2)


pmod(id, 2)

### Выводы
+ Spark обладает широким набором функций для работы с колонками разных типов, включая простые типы - строки, числа, и т. д., а также словари, массивы и структуры
+ Встроенные функции принимают колонки `org.apache.spark.sql.Column` и возвращают `org.apache.spark.sql.Column` в большинстве случаев
+ Встроенные функции доступны в двух местах - org.apache.spark.sql.functions и SQL built-in functions
+ Встроенные функции можно (и нужно) использовать вместе - на вход во встроенные функции могут подаваться результаты встроенной функции, тк все они возвращают `sql.Column` 

### Пользовательские функции

В том случае, если функционала встроенных функций не хватает, можно написать пользовательскую функцию - UDF. Пользовательская функция может принимать до 16 аргументов. Соответствие Spark и Scala типов описано [здесь](https://spark.apache.org/docs/latest/sql-reference.html#data-types)

Необходимо помнить, что `null` в Spark превращается в `null` внутри UDF

In [100]:
import org.apache.spark.sql.functions.{udf, col}

val df = spark.range(0,10)

val plusOne = udf { (value: Long, v2: Long, v3: Long) => value + v2 + v3 }

df.withColumn("idPlusOne", plusOne(col("id"), col("id"), lit(10+2+2))).show()
df.withColumn("idPlusOne", plusOne(col("id"), col("id"), lit(10+2+2))).explain(true)

+---+---------+
| id|idPlusOne|
+---+---------+
|  0|       14|
|  1|       16|
|  2|       18|
|  3|       20|
|  4|       22|
|  5|       24|
|  6|       26|
|  7|       28|
|  8|       30|
|  9|       32|
+---+---------+

== Parsed Logical Plan ==
'Project [id#1920L, UDF('id, 'id, 14) AS idPlusOne#1932]
+- Range (0, 10, step=1, splits=Some(2))

== Analyzed Logical Plan ==
id: bigint, idPlusOne: bigint
Project [id#1920L, if (((isnull(id#1920L) || isnull(id#1920L)) || isnull(cast(14 as bigint)))) null else UDF(id#1920L, id#1920L, cast(14 as bigint)) AS idPlusOne#1932L]
+- Range (0, 10, step=1, splits=Some(2))

== Optimized Logical Plan ==
Project [id#1920L, UDF(id#1920L, id#1920L, 14) AS idPlusOne#1932L]
+- Range (0, 10, step=1, splits=Some(2))

== Physical Plan ==
*(1) Project [id#1920L, UDF(id#1920L, id#1920L, 14) AS idPlusOne#1932L]
+- *(1) Range (0, 10, step=1, splits=2)


df = [id: bigint]
plusOne = UserDefinedFunction(<function3>,LongType,Some(List(LongType, LongType, LongType)))


UserDefinedFunction(<function3>,LongType,Some(List(LongType, LongType, LongType)))

Пользовательская функция может возвращать:
+ простой тип - `String`, `Long`, `Float`, `Boolean` и т.д.
+ массив - любые коллекции, наследующие `Seq[T]` - `List[T]`, `Vector[T]` и т. д.
+ словарь - `Map[A,B]`
+ инстанс `case class`'а
+ Option[T]

Реализуем функцию, которая возвращает имя хоста, на котором работает воркер:

In [101]:
import java.net.InetAddress

val hostname = udf { () => InetAddress.getLocalHost().getHostName() }

df.withColumn("hostname", hostname()).show(10, false)

+---+--------------+
|id |hostname      |
+---+--------------+
|0  |spark-master-3|
|1  |spark-master-3|
|2  |spark-master-3|
|3  |spark-master-3|
|4  |spark-master-3|
|5  |spark-master-3|
|6  |spark-master-3|
|7  |spark-master-3|
|8  |spark-master-3|
|9  |spark-master-3|
+---+--------------+



hostname = UserDefinedFunction(<function0>,StringType,Some(List()))


UserDefinedFunction(<function0>,StringType,Some(List()))

Мы также можем использовать монады `Try[T]` и `Option[T]` и для написания пользовательской функции:

In [102]:
import scala.util.Try
import org.apache.spark.sql.functions.{udf, col}

val df = spark.range(0,10)

val divideTwoBy = udf { (inputValue: Long) => Try(2L / inputValue).toOption }

val result = df.withColumn("divideTwoBy", divideTwoBy(col("id")))
result.printSchema
result.show(10, false)

root
 |-- id: long (nullable = false)
 |-- divideTwoBy: long (nullable = true)

+---+-----------+
|id |divideTwoBy|
+---+-----------+
|0  |null       |
|1  |2          |
|2  |1          |
|3  |0          |
|4  |0          |
|5  |0          |
|6  |0          |
|7  |0          |
|8  |0          |
|9  |0          |
+---+-----------+



df = [id: bigint]
divideTwoBy = UserDefinedFunction(<function1>,LongType,Some(List(LongType)))
result = [id: bigint, divideTwoBy: bigint]


[id: bigint, divideTwoBy: bigint]

### Выводы
+ Пользовательские функции позволяют реализовать произвольный алгоритм и использовать его в DF API
+ Пользовательские функции работают медленнее встроенных, поскольку при использовании встроенных функций Spark использует ряд оптимизаций, например векторизацию вычислений на уровне CPU

## Соединения

Join'ы позволяют соединять два DF в один по заданным условиям.

По типу условия join'ы делятся на:
+ equ-join - соединение по равенству одного или более ключей
+ non-equ join - соединение по условию, отличному от равенства одного или более ключей

#### Виды соединений
+ **BroadcastHashJoin**
  - equ join
  - broadcast
+ **SortMergeJoin**
  - equ join
  - sortable keys
+ **BroadcastNestedLoopJoin**
  - non-equ join
  - using broadcast
+ **CartesianProduct**
  - non-equ join

Добавим новую колонку к датасету `airports`, в которой будет процент заданного типа аэропорта ко всем типам аэропорта по каждой стране. Первым шагом посчитаем число аэропортов каждого типа по стране:

In [103]:
import org.apache.spark.sql.functions.{count, round, lit}

val aggTypeCountry = airports.groupBy('type, 'iso_country).agg(count("*").alias("cnt_country_type"))

aggTypeCountry.show(5, false)

+--------------+-----------+----------------+
|type          |iso_country|cnt_country_type|
+--------------+-----------+----------------+
|small_airport |MP         |1               |
|large_airport |GB         |27              |
|heliport      |CH         |19              |
|closed        |LT         |4               |
|medium_airport|SS         |3               |
+--------------+-----------+----------------+
only showing top 5 rows



aggTypeCountry = [type: string, iso_country: string ... 1 more field]


[type: string, iso_country: string ... 1 more field]

Теперь посчитаем количество аэропортов по каждой стране:

In [104]:
val aggCountry = airports.groupBy('iso_country).agg(count("*").alias("cnt_country"))
aggCountry.show(5, false)

+-----------+-----------+
|iso_country|cnt_country|
+-----------+-----------+
|MM         |75         |
|DZ         |61         |
|LT         |59         |
|CI         |26         |
|TC         |8          |
+-----------+-----------+
only showing top 5 rows



aggCountry = [iso_country: string, cnt_country: bigint]


[iso_country: string, cnt_country: bigint]

Соединим получившиеся датасеты и получим процентное распределение типов аэропорта по стране

In [105]:
val percent = 
    aggTypeCountry
        .join(aggCountry, Seq("iso_country"), "inner")
        .select('iso_country, 'type, (round(lit(100) * 'cnt_country_type / 'cnt_country, 2).alias("percent")))
percent.show(5, false)

+-----------+--------------+-------+
|iso_country|type          |percent|
+-----------+--------------+-------+
|MP         |small_airport |9.09   |
|GB         |large_airport |2.24   |
|CH         |heliport      |21.84  |
|LT         |closed        |6.78   |
|SS         |medium_airport|6.52   |
+-----------+--------------+-------+
only showing top 5 rows



percent = [iso_country: string, type: string ... 1 more field]


[iso_country: string, type: string ... 1 more field]

Соединим полученный датасет с изначальным:

In [106]:
val result = airports.join(percent, Seq("iso_country", "type"), "left")
result.select('ident, 'iso_country, 'type, 'percent).sample(0.2).show(20, false)

+-------+-----------+--------------+-------+
|ident  |iso_country|type          |percent|
+-------+-----------+--------------+-------+
|EGAC   |GB         |large_airport |2.24   |
|EGCN   |GB         |large_airport |2.24   |
|EGHI   |GB         |large_airport |2.24   |
|EGNX   |GB         |large_airport |2.24   |
|EGSS   |GB         |large_airport |2.24   |
|EGTE   |GB         |large_airport |2.24   |
|EGUL   |GB         |large_airport |2.24   |
|EGVN   |GB         |large_airport |2.24   |
|TT01   |MP         |small_airport |9.09   |
|LSER   |CH         |heliport      |21.84  |
|LSHC   |CH         |heliport      |21.84  |
|LSHG   |CH         |heliport      |21.84  |
|LSXL   |CH         |heliport      |21.84  |
|LSXR   |CH         |heliport      |21.84  |
|LSXW   |CH         |heliport      |21.84  |
|HSFA   |SS         |medium_airport|6.52   |
|ZLG    |EH         |closed        |16.67  |
|MHCT   |HN         |closed        |3.16   |
|MD-0001|MD         |closed        |10.0   |
|SPIM   |P

result = [iso_country: string, type: string ... 11 more fields]


[iso_country: string, type: string ... 11 more fields]

Во всех наших джойнах присутствует массив `Seq[String]`. Это синтаксических сахар, позволяющий не переименовывать колонки датасетов, а просто указать, что соединение будет делаться по колонкам с именами, входящим в массив.

В общем случае условие джойна должно быть выражено в виде колонки `sql.Column`, например:

In [114]:
import org.apache.spark.sql.Column
val joinCondition: Column = (airports("iso_country") === percent("iso_country")) and (airports("type") === percent("type"))

joinCondition = ((iso_country = iso_country) AND (type = type))


((iso_country = iso_country) AND (type = type))

In [115]:
val result2 = airports.join(percent, joinCondition, "left")
result2.select(airports("ident"), airports("iso_country"), airports("type"), percent("percent")).sample(0.2).show(20, false)
result2.select(airports("ident"), airports("iso_country"), airports("type"), percent("percent")).sample(0.2).explain()

+-----+-----------+--------------+-------+
|ident|iso_country|type          |percent|
+-----+-----------+--------------+-------+
|EGGD |GB         |large_airport |2.24   |
|EGGW |GB         |large_airport |2.24   |
|EGHH |GB         |large_airport |2.24   |
|EGKK |GB         |large_airport |2.24   |
|EGNM |GB         |large_airport |2.24   |
|TT01 |MP         |small_airport |9.09   |
|LSHG |CH         |heliport      |21.84  |
|LSXR |CH         |heliport      |21.84  |
|LSXW |CH         |heliport      |21.84  |
|LSXY |CH         |heliport      |21.84  |
|SPZO |PE         |large_airport |1.14   |
|LOWG |AT         |medium_airport|4.79   |
|EBCV |BE         |medium_airport|5.48   |
|EBFS |BE         |medium_airport|5.48   |
|HEAL |EG         |medium_airport|37.14  |
|HEAR |EG         |medium_airport|37.14  |
|HEBA |EG         |medium_airport|37.14  |
|HEGS |EG         |medium_airport|37.14  |
|HEMA |EG         |medium_airport|37.14  |
|HESC |EG         |medium_airport|37.14  |
+-----+----

result2 = [ident: string, type: string ... 13 more fields]


[ident: string, type: string ... 13 more fields]

При этом в данном выражении допускается использование встроенных функций, пользовательских функций и операторов сравнения. Однако следует помнить, что мы выполняем джойн двух распределенных датасетов и если условие соединения будет плохо составлено, то Spark выполнит `cross join`, производительность которого будет "крайне мала" &copy;

### Выводы:
+ Spark поддерживает большое число типов соединений
+ Условием соединения может быть `Seq[String]`, либо `sql.Column`
+ При использовании сложных условий соединения следует избегать тех, которые приведут к `cross join`

## Оконные функции
Оконные функции позволяют делать функции над "окнами" (кто бы мог подумать) данных

Окно создается из класса [org.apache.spark.sql.expressions.Window](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.expressions.Window) с указанием полей, определяющих границы окон и полей, определяющих порядок сортировки внутри окна:

```val window = Window.partitionBy("a", "b").orderBy("a")```

Применяя окна, можно использовать такие полезные функции из [org.apache.spark.sql.functions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$), как ```lag()``` и ```lead()```, а также эффективно работать с данными time-series данными.

Выполним задачу с вычисление процента отношения типов аэропортов, используя оконные функции.

In [122]:
lit(1).as("a")

1 AS `a`

In [123]:
import org.apache.spark.sql.expressions.Window

val w = Window.partitionBy()
val windowCountry = Window.partitionBy("iso_country")
val windowTypeCountry = Window.partitionBy("type", "iso_country")

val result = airports
                .withColumn("cnt_country", count("*").over(windowCountry))
                .withColumn("cnt_country_type", count("*").over(windowTypeCountry))
                .withColumn("cnt_all", count(lit(1)).over(w))
                .withColumn("percent", round(lit(100) * 'cnt_country_type / 'cnt_country, 2))
                            
result.select('ident, 'iso_country, 'type, 'percent, 'cnt_all).sample(0.2).show(20, false)
result.select('ident, 'iso_country, 'type, 'percent, 'cnt_all).sample(0.2).explain()

Name: Unknown Error
Message: <console>:88: error: overloaded method value count with alternatives:
  (columnName: String)org.apache.spark.sql.TypedColumn[Any,Long] <and>
  (e: org.apache.spark.sql.Column)org.apache.spark.sql.Column
 cannot be applied to (Int)
                       .withColumn("cnt_all", count(1).over(w))
                                              ^

StackTrace: 

### Выводы:
+ Оконные функции позволяют применять функции, применительно к окнам данных
+ Окно определяется списком колонок и сортировкой
+ Применение оконных функций приводит к `shuffle`

После завершения работы не забывайте останавливать `SparkSession`, чтобы освободить ресурсы кластера!

In [ ]:
spark.stop